<a href="https://colab.research.google.com/github/bikash-bhoi/eva4/blob/master/%20Session15/Session15A_Mask_RCNN01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import os

In [0]:
#copy bg/fg/mask files from Drive
%%bash
cd /content
mkdir Mask_RCNN
cp -r /content/gdrive/My\ Drive/Mask_RCNN/* Mask_RCNN/

In [4]:
%%bash
ls Mask_RCNN/Cars_Fg/ |wc -l
ls Mask_RCNN/Cars_Mask/ |wc -l
ls Mask_RCNN/background/ |wc -l
ls Mask_RCNN/fg_bg/ |wc -l
ls Mask_RCNN/fg_bg_mask/ |wc -l

100
100
100
0
0


In [0]:
## Function to Create Overlay
def overlay_fg_bg(fg,bg,x_offset,y_offset,tgt_save):
    fg_bg=bg.copy()
    y1, y2 = y_offset, y_offset + fg.shape[0]
    x1, x2 = x_offset, x_offset + fg.shape[1]
    
    alpha_fg = fg[:, :, 3] / 255.0
    alpha_bg = 1.0 - alpha_fg

    for c in range(0, 3):
        fg_bg[y1:y2, x1:x2, c] = (alpha_fg * fg[:, :, c] +
                                  alpha_bg * bg[y1:y2, x1:x2, c])


    cv2.imwrite(tgt_save,fg_bg)
    
    #arr = np.asarray(fg_bg)
    #plt.imshow(arr, vmin=0, vmax=255)
    #plt.show()
    
def overlay_mask(fg,bg,x_offset,y_offset,tgt_save):
    fg_bg=bg.copy()
    fg_bg[y_offset:y_offset+fg.shape[0], x_offset:x_offset+fg.shape[1]] = fg

    cv2.imwrite(tgt_save,fg_bg)

In [0]:
tgt_path="/content/gdrive/My Drive/Mask_RCNN/fg_bg/"
mask_tgt_path="/content/gdrive/My Drive/Mask_RCNN/fg_bg_mask/"
bg_path="./Mask_RCNN/background/"
fg_path="./Mask_RCNN/Cars_Fg/"
mask_path="./Mask_RCNN/Cars_Mask/"

mask_bg = np.zeros((200,200), np.uint8)

for i_bg in os.listdir(bg_path):
    bg = cv2.imread(bg_path+i_bg)
    bg_name=os.path.splitext(i_bg)[0]
    
    for i_fg in os.listdir(fg_path):
        fg = cv2.imread(fg_path+i_fg, -1)
        fg_name=os.path.splitext(i_fg)[0]
        mask_fg = cv2.imread(mask_path+i_fg, -1)
        fg_flip=cv2.flip(fg,1)
        fg_flip_name=fg_name+'_flip'
        mask_fg_flip=cv2.flip(mask_fg,1)
        for i in range(1,21):
            x_offset=randint(20,116)
            y_offset=randint(20,132)
            #fg on bg
            tgt_save=tgt_path+bg_name+fg_name+'-'+str(i)+'.jpg'
            overlay_fg_bg(fg,bg,x_offset,y_offset,tgt_save)
            
            #mask on black bg
            tgt_save=mask_tgt_path+bg_name+fg_name+'-'+str(i)+'.jpg'
            overlay_mask(mask_fg,mask_bg,x_offset,y_offset,tgt_save)

            #LR Flipped fg on bg
            tgt_save=tgt_path+bg_name+fg_flip_name+'-'+str(i)+'.jpg'
            overlay_fg_bg(fg_flip,bg,x_offset,y_offset,tgt_save)
            
            #LR Flipped mask on black bg
            tgt_save=mask_tgt_path+bg_name+fg_flip_name+'-'+str(i)+'.jpg'
            overlay_mask(mask_fg_flip,mask_bg,x_offset,y_offset,tgt_save)

In [7]:
%%bash
##Copy overlay fg_bg and mask images to Drive
cp ./Mask_RCNN/fg_bg/* /content/gdrive/My\ Drive/Mask_RCNN/fg_bg/
cp ./Mask_RCNN/fg_bg_mask/* /content/gdrive/My\ Drive/Mask_RCNN/fg_bg_mask/

bash: line 2: /bin/cp: Argument list too long
bash: line 3: /bin/cp: Argument list too long


In [9]:
##Copy overlay fg_bg and mask images to Drive
!find  ./Mask_RCNN/fg_bg/ -name "*.jpg" -exec cp -uf "{}" /content/gdrive/My\ Drive/Mask_RCNN/fg_bg/ \;
!find  ./Mask_RCNN/fg_bg_mask/ -name "*.jpg" -exec cp -uf "{}" /content/gdrive/My\ Drive/Mask_RCNN/fg_bg_mask/ \;

^C


In [10]:
!find  ./Mask_RCNN/fg_bg_mask/ -name "*.jpg" -exec cp -uf "{}" /content/gdrive/My\ Drive/Mask_RCNN/fg_bg_mask/ \;

^C
